In [ ]:
!pip install pytorch_lightning 

     |████████████████████████████████| 523 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 48.8 MB/s 
     |████████████████████████████████| 329 kB 42.9 MB/s 
     |████████████████████████████████| 596 kB 35.2 MB/s 
     |████████████████████████████████| 829 kB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 35.8 MB/s 
     |████████████████████████████████| 271 kB 47.1 MB/s 
     |████████████████████████████████| 160 kB 45.3 MB/s 
     |████████████████████████████████| 192 kB 48.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d179aabb1933b1778433a956bf3736f409fd15c526b13d44d253787824074802
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found exis

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install -q wandb
import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 140 kB 41.7 MB/s 
     |████████████████████████████████| 180 kB 46.3 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import zipfile
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import pytorch_lightning as pl 
from torchvision import models 
from torchvision import transforms as T
import torch
%matplotlib inline

In [ ]:
def prepare_data():
    zip = zipfile.ZipFile('custom_data.zip')
    fnames = [fnames for fnames in zip.namelist() if 'apple' in fnames]
    print('Loading images metadata...')
    images = [Image.open(zip.open(fnames)) for fnames in fnames]
    print('Reading pixels...')
    for i, img in enumerate(images):
        img.load()
        if i % 100 == 0:
            print(i)
    [img.load() for img in images]
    print('Done', len(images))
    targets = [1 if 'dog' in fnames else 0 for fnames in fnames]

    return images, targets

In [ ]:
class MyDataset(Dataset):
  def __init__(self, images, targets):
    super().__init__()
    self.images = images
    self.targets = targets

  def __len__(self):
    return len(self.targets)

  def __getitem__(self, idx):
    img = self.images[idx]
    label  = self.targets[idx]
    
    return img, label 


In [ ]:
class Mynet(pl.LightningModule):
  def _init_(self):
    super()._init_()
    self.features = models.resnet18(pretrained=True)
    self.features.fc = torch.nn.Linear(512, 1)

  def forward(self, x):
    out = self.features(x)
    return out
  
  def training_step(self, batch, batch_idx):
    x, y = batch
    out = self(x)
    loss = nn.CrossEntropyLoss(out, y)
    self.log("train_loss", loss)
    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    out = self(x)
    loss = nn.CrossEntropyLoss()(out, y)
    _, preds = torch.max(out, 1)
    self.log("val_loss", loss)
    self.log("val_acc", (preds==y).sum()/len(y))
    return preds
  
  def configure_optimizers(self):
    return torch.optim.Adam(self.parameters(), lr=1e-3)


In [ ]:
class MyDataModule(pl.LightningDataModule):
  def __init__(self, images, targets, test_size=0.3):
    super().__init__()
    self.images = images
    self.targets = targets
    self.test_size = test_size
  
  def setup(self, stage =None):
    train_names = self.targets[:40]
    val_names = self.targets[40:]
    self.train = MyDataset(self.images, train_names)
    self.val = MyDataset(self.images, val_names)

  def train_dataloader(self):
      return DataLoader(self.train, batch_size=16)

  def val_dataloader(self):
      return DataLoader(self.val, batch_size=16)

In [ ]:
def main():
  net = Mynet()
  images, targets = prepare_data()
  dm = MyDataModule(images, targets)
  trainer = pl.Trainer(gpus=1, max_epochs=1)
  trainer.fit(net, dm)
  return net

net = main()